In [0]:
# Lister toutes les bases de données
spark.sql("SHOW DATABASES").show()

# Sélectionner ta base de données (exemple : default)
spark.sql("USE default")

# Lister les tables disponibles dans la base
spark.sql("SHOW TABLES").show()


In [0]:
# Charger les bibliothèques Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit


spark = SparkSession.builder.appName("NettoyageDonnees").getOrCreate()

# Charger les tables
df1 = spark.table("default.yellow_tripdata_2024_01")
df2 = spark.table("default.yellow_tripdata_2024_03")
df3 = spark.table("default.yellow_tripdata_2024_07")
df4 = spark.table("default.yellow_tripdata_2024_11")
df5 = spark.table("default.yellow_tripdata_2025_09")


# Trouver toutes les colonnes présentes dans au moins une table
all_columns = set(df1.columns) | set(df2.columns) | set(df3.columns) | set(df4.columns) | set(df5.columns)

# Fonction pour ajouter les colonnes manquantes
def add_missing_cols(df, all_cols):
    existing_cols = set(df.columns)
    for c in all_cols - existing_cols:
        df = df.withColumn(c, lit(None))  # ajoute une colonne vide (null)
    return df.select(sorted(all_cols))  # garder le même ordre

# Ajouter les colonnes manquantes
df1 = add_missing_cols(df1, all_columns)
df2 = add_missing_cols(df2, all_columns)
df3 = add_missing_cols(df3, all_columns)
df4 = add_missing_cols(df4, all_columns)
df5 = add_missing_cols(df5, all_columns)


merged_df1 = df1.union(df2)
merged_df2 = merged_df1.union(df3)
merged_df3 = merged_df2.union(df4)
df = merged_df3.union(df5)

df.write.mode("overwrite").saveAsTable("default.Raw_table")

# Afficher les 5 premières lignes
df.show(5)
